In [1]:
import torch
from byol_pytorch import BYOL
from torchvision import models
from PIL import Image
from torch.utils.data import DataLoader,Dataset
import torchvision.transforms as transforms
import glob
import os
from tqdm import tqdm


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
filenameToPILImage = lambda x: Image.open(x)
class mini_img(Dataset):
    def __init__(self, path):
        self.imgpaths = glob.glob(os.path.join(path,'*.jpg'))
        self.transform = transforms.Compose([
            filenameToPILImage,
            transforms.Resize(128),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])

    def __getitem__(self, index):
        image = self.transform(self.imgpaths[index])
        return image

    def __len__(self):
        return len(self.imgpaths)

In [4]:
mini_dataset = mini_img('hw4_data/mini/train')
mini_dataloader = DataLoader(mini_dataset, batch_size=64, shuffle=True)

In [5]:
resnet = models.resnet50(pretrained=False).to(device)
resnet.load_state_dict(torch.load('new/6_0.7097improved-net.pt'))
learner = BYOL(
    resnet,
    image_size = 128,
    use_momentum = False,
    hidden_layer = 'avgpool'
)

opt = torch.optim.Adam(learner.parameters(), lr=3e-4)

In [6]:
def mini_train(model, epoch):
    for ep in range(epoch):
        loss_ = 0
        total = 0
        for i, img in enumerate(tqdm(mini_dataloader)):
            loss = model(img.to(device))
            loss_ += loss.item()
            total += img.size(0)
            opt.zero_grad()
            loss.backward()
            opt.step()
            # model.update_moving_average()
        print(f'Loss:{loss_/total:.4f}')
        torch.save(resnet.state_dict(), f'./new/{ep+6}_{loss:.4f}improved-net.pt')
# save your improved network

In [7]:
mini_train(learner, 100)
# torch.save(resnet.state_dict(), './improved-net.pt')

100%|██████████| 600/600 [22:11<00:00,  2.22s/it]


Loss:0.0115


100%|██████████| 600/600 [22:02<00:00,  2.20s/it]


Loss:0.0090


100%|██████████| 600/600 [22:10<00:00,  2.22s/it]


...

Loss:0.0052


100%|██████████| 600/600 [21:44<00:00,  2.17s/it]


Loss:0.0054


  3%|▎         | 20/600 [00:45<21:48,  2.26s/it]


KeyboardInterrupt: 